In [6]:
%pip install scikit-learn scipy matplotlib numpy

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [8]:
%pip install numpy --upgrade

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 5.0 MB/s eta 0:00:000m eta 0:00:010:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
%pip install protobuf==3.20.5

Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement protobuf==3.20.5 (from versions: 2.0.0b0, 2.0.3, 2.3.0, 2.4.1, 2.5.0, 2.6.0, 2.6.1, 3.0.0a2, 3.0.0a3, 3.0.0b1, 3.0.0b1.post1, 3.0.0b1.post2, 3.0.0b2, 3.0.0b2.post1, 3.0.0b2.post2, 3.0.0b3, 3.0.0b4, 3.0.0, 3.1.0, 3.1.0.post1, 3.2.0rc1, 3.2.0rc1.post1, 3.2.0rc2, 3.2.0, 3.3.0, 3.4.0, 3.5.0.post1, 3.5.1, 3.5.2, 3.5.2.post1, 3.6.0, 3.6.1, 3.7.0rc2, 3.7.0rc3, 3.7.0, 3.7.1, 3.8.0rc1, 3.8.0, 3.9.0rc1, 3.9.0, 3.9.1, 3.9.2, 3.10.0rc1, 3.10.0, 3.11.0rc1, 3.11.0rc2, 3.11.0, 3.11.1, 3.11.2, 3.11.3, 3.12.2, 3.12.4, 3.13.0rc3, 3.13.0, 3.14.0rc1, 3.14.0rc2, 3.14.0rc3, 3.14.0, 3.15.0rc1, 3.15.0rc2, 3.15.0, 3.15.1, 3.15.2, 3.15.3, 3.15.4, 3.15.5, 3.15.6, 3.15.7, 3.15.8, 3.16.0rc1, 3.16.0rc2, 3.16.0, 3.17.0rc1, 3.17.0rc2, 3.17.0, 3.17.1, 3.17.2, 3.17.3, 3.18.0rc1, 3.18.0rc2, 3.18.0, 3.18.1, 3.18.3, 3.19.0rc1, 3.19.0rc2, 3.19.0, 3.19.1, 3.19.2, 3.19.3, 3.19.4, 3.19.5

In [2]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import LSTM, Dense, Flatten
from functions import VM
import os
import numpy as np

2023-04-13 17:10:28.867776: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-13 17:10:28.869705: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-13 17:10:28.905167: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-13 17:10:28.905906: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-13 17:10:29.598766: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

# Take the sign names in our collections data

In [63]:
signs = sorted(os.listdir("./Keypoint_Data/")) # take all the signs used in data collection
sign_id = {label: num for num, label in enumerate(signs)} # assign a value for each sign (each sign is assign to its index)

In [64]:
signs[0]

'hello'

In [65]:
sign_id['hello']

0

# Create the our x and y data


In [66]:
sequences, labels = [], [] # sequences represents our x data while labels represents our y data
# create a loop that creates our x and y data
for sign in signs:
    for sequence in range(VM.seq):
        window = []
        for frames in range(VM.seq_lenght):
            res = np.load(os.path.join(VM.DATA_PATH, sign, str(sequence), f"{frames}.npy"))
            window.append(res)
        sequences.append(window)
        labels.append(sign_id[sign])
        
# store our x and y data to a numpy array
x_data = np.array(sequences) 
y_data = to_categorical(labels).astype(int) 

# setup our training and testing partition
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.05)

In [41]:
x_train.shape
x_train.shape[1:]

(30, 1530)

# Build and train the model


In [67]:
# build the model
model = Sequential()
model.add(LSTM(64, return_sequences="True", activation="relu", input_shape=(30,1530)))
model.add(LSTM(128, return_sequences="True", activation="relu"))
model.add(LSTM(64, return_sequences="False", activation="relu"))
model.add(Flatten(input_shape=(x_train.shape[1:])))
model.add(Dense(64, activation="relu"))
model.add(Dense(32, activation="relu"))
model.add(Dense(np.array(signs).shape[0], activation="softmax"))

In [49]:
model.compile(optimizer="Adam", loss="categorical_crossentropy", metrics=["categorical_accuracy"]) # compiler 

In [50]:
model.fit(x_train, y_train, epochs=175) # run this line when data collection is done (this will train the model)
# we can start the number of epochs to be equal to 175 to avoid over training and add afterwards if needed

Epoch 1/175
15/15 [==============================] - 5s 54ms/step - loss: 2.3451 - categorical_accuracy: 0.1095
Epoch 2/175
15/15 [==============================] - 1s 57ms/step - loss: 1.9730 - categorical_accuracy: 0.2674
Epoch 3/175
15/15 [==============================] - 1s 54ms/step - loss: 2.0401 - categorical_accuracy: 0.2379
Epoch 4/175
15/15 [==============================] - 1s 54ms/step - loss: 1.7355 - categorical_accuracy: 0.3474
Epoch 5/175
15/15 [==============================] - 1s 54ms/step - loss: 1.4769 - categorical_accuracy: 0.3937
Epoch 6/175
15/15 [==============================] - 1s 54ms/step - loss: 2.2201 - categorical_accuracy: 0.2084
Epoch 7/175
15/15 [==============================] - 1s 54ms/step - loss: 2.2872 - categorical_accuracy: 0.1474
Epoch 8/175
15/15 [==============================] - 1s 55ms/step - loss: 2.1238 - categorical_accuracy: 0.1937
Epoch 9/175
15/15 [==============================] - 1s 54ms/step - loss: 2.0426 - categorical_accuracy:

15/15 [==============================] - 1s 54ms/step - loss: 0.4533 - categorical_accuracy: 0.8189
Epoch 74/175
15/15 [==============================] - 1s 55ms/step - loss: 0.4304 - categorical_accuracy: 0.8400
Epoch 75/175
15/15 [==============================] - 1s 54ms/step - loss: 0.3191 - categorical_accuracy: 0.8821
Epoch 76/175
15/15 [==============================] - 1s 54ms/step - loss: 0.2075 - categorical_accuracy: 0.9411
Epoch 77/175
15/15 [==============================] - 1s 54ms/step - loss: 0.1881 - categorical_accuracy: 0.9495
Epoch 78/175
15/15 [==============================] - 1s 53ms/step - loss: 0.1682 - categorical_accuracy: 0.9389
Epoch 79/175
15/15 [==============================] - 1s 56ms/step - loss: 0.1726 - categorical_accuracy: 0.9411
Epoch 80/175
15/15 [==============================] - 1s 53ms/step - loss: 0.2943 - categorical_accuracy: 0.8968
Epoch 81/175
15/15 [==============================] - 1s 54ms/step - loss: 0.3482 - categorical_accuracy: 0.8

KeyboardInterrupt: 

In [62]:
del model

In [51]:
model.summary() # summary of our model

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_12 (LSTM)              (None, 30, 64)            408320    
                                                                 
 lstm_13 (LSTM)              (None, 30, 128)           98816     
                                                                 
 lstm_14 (LSTM)              (None, 30, 64)            49408     
                                                                 
 flatten_4 (Flatten)         (None, 1920)              0         
                                                                 
 dense_12 (Dense)            (None, 64)                122944    
                                                                 
 dense_13 (Dense)            (None, 32)                2080      
                                                                 
 dense_14 (Dense)            (None, 10)               

In [52]:
res = model.predict(x_test)

1/1 [==============================] - 0s 387ms/step


In [60]:
signs[np.argmax(res[0])]

'hi'

In [61]:
signs[np.argmax(y_test[0])]

'wait'

In [55]:
y_test

array([[0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

# save weights


In [16]:
model.save("try.h5") # save model

In [17]:
model = tf.keras.models.load_model("./try.h5")
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [
  tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
  tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
]
tflite = converter.convert()

open("./try.tflite", "wb").write(tflite)

INFO:tensorflow:Assets written to: C:\Users\Baron\AppData\Local\Temp\tmpvxtgdg4q\assets


652552

In [38]:
interpreter = tf.lite.Interpreter("LiteML.tflite")

inp = interpreter.get_input_details()
out = interpreter.get_output_details()

interpreter.set_tensor(inp[0]["index"], np.expand_dims(sequences,axis=0))
interpreter.invoke()
                
                #for i in sequence[-30:]:
output_data = interpreter.get_tensor(out[0]["index"])
                
highest = np.argmax(output_data)
word = VM.get_key(signs, highest)
print(output_data.shape)
print(word)


ValueError: Cannot set tensor: Got value of type FLOAT64 but expected type FLOAT32 for input 0, name: serving_default_lstm_9_input:0 